In [13]:
import numpy as np
import sys, os

In [14]:
sys.path.append("..")
from qgbaroclinic import OceBaroclinicModes
from qgbaroclinic.plot import PlotMap

In [ ]:
"""
Example for computing the MAP of rossby radii in a selected region.
"""

In [ ]:
# Define Ocean Baroclinic Modes object;
# "latitude", "longitude" are the dimension names in OCEAN NetCDF file.
obm = OceBaroclinicModes(longitude=[-30, -29], latitude=[40, 41])

In [ ]:
current_path = os.path.dirname(__vsc_ipynb_file__)
path_to_my_data = os.path.join(current_path, "../my_data/reanalysis")
# Extract OCEAN variables form NetCDF file.
temp, sal, depth, lat , lon = obm.read(
    path_to_my_data,
    "thetao",
    "so",
    "depth",
    "latitude",
    "longitude",
)

In [ ]:
# Compute Potential Density and average in time.
pot_density = obm.potential_density(temp, sal)
mean_pot_density = np.nanmean(pot_density, axis = 0)

In [ ]:
elevation = np.ones_like(mean_pot_density) * 3000 # dummy elevation

In [ ]:
n_modes = 3
lat_size, lon_size = mean_pot_density.shape[1:]
rossby_rad = np.empty([lat_size, lon_size, n_modes])

In [ ]:
# Compute Rossby Baroclinic Deformation Radii.
for i in range(lat_size):
    print(f"Processing row {i}/{lat_size}")
    for j in range(lon_size):
        try:
            rossby_rad[i,j,:], _ = obm(mean_pot_density[:,i,j], depth.values, lat[i].values, elevation[i,j])
        except ValueError:
            pass

In [ ]:
# Plot and compare with Chelton (1998)
pm = PlotMap([-30, -28], [40, 42], ".")
pm.make_plot(rossby_rad[:,:,1]/1000,lon, lat, "2d_map_example", offset=0.0)
pm.plot_chelton_map()

In [ ]:
"""
NOTE: the results are different from Chelton (1998) since we are using a dummy bathymetry.
"""